# membrane-toolkit core functions demo

This notebook will demonstrate some of the capabilities of membrane-toolkit's core functions.

## Setup
---

**NOTE:** when importing, you must use and underscore instead of a hyphen in 'membrane_toolkit'

In [1]:
from membrane_toolkit.core import donnan_equilibrium

## Main code
---

### A simple calculation

Let's use membrane-toolkit to perform a simple Donnan exclusion calculation. The Donnan equilibrium between a membrane with fixed charged concentration $\bar C_{fix}$ (mol per L water sorbed) and a salt solution of bulk concentration $C_s$ (mol/L) is given by:

$$
\bar C_{co}^{\nu_{co}} \big ( \frac{z_{co} \bar C_{co} + z_{fix} \bar C_{fix}}{z_{ct}} \big )^{\nu_{ct}} = - \Gamma \nu_{ct}^{\nu_{ct}} \nu_{co}^{\nu_{co}}  C_s^{\nu_{ct} + \nu_{co}}
$$

where subscripts $co$ and $ct$ indicate the co-ion (same charge as the membrane) and counter-ion (opposite charge to the membrane), $\nu$ (dimensionless) are stoichiometric coefficients, and overbars indicate membrane-phase quantities, units of moles per liter of water sorbed by the membrane. $\Gamma$ (dimensionless) is the ratio of activity coefficients in the bulk solution to those in the membrane, given by:

$$
\Gamma = \frac{\gamma_{\pm}^{\nu_{ct} + \nu_{co}}}{\bar \gamma_{ct}^{\nu_{ct}} \bar \gamma_{co}^{\nu_{co}}}
$$

Of course, it is much more common in literature to see a simplified form of the Donnan equilibrium, which applies to monovalent, 1:1 salts when $z_{fix}$ = 1, and $\Gamma$ is assumed to equal 1. Subject to those assumptions, the above equations simplify to

$$
\bar C_{co} = C_s \exp \big(-asinh( \frac{\bar C_{fix}}{2 C_s })\big )
$$

First, let's calculate the co-ion concentration using the `donnan_equilibrium` function, and see if it compares to our expectations from the simplified formula. Consider a bulk NaCl solution with $C_s$=0.5 mol/L and a membrane with $\bar C_{fix}$= 4 mol/L

In [2]:
from membrane_toolkit.core import donnan_equilibrium
# assuming a monovalent 1:1 salt, we only need to supply two arguments
Cs = 0.5
Cfix = 4.0

Cco = donnan_equilibrium(Cs, Cfix)
print("{:.3f} mol/L".format(Cco))

0.062 mol/L


Let's compare this to the result we could calculate manually via the simplified formula

In [3]:
import numpy as np
Cco = Cs * np.exp(-1*np.arcsinh(Cfix / 2 / Cs))
print("{:.3f} mol/L".format(Cco))

0.062 mol/L


If we have a more complicated situation (e.g., a multivalent salt), we simply add additional arguments to represent the stoichiometry:

In [4]:
from membrane_toolkit.core import donnan_equilibrium
# for MgCl2, we need to supply additional arguments z_counter = +2, nu_co = 2, z_co = -1, and nu_co = 2
Cs = 0.5
Cfix = 4.0

Cco = donnan_equilibrium(Cs, Cfix, z_counter=2, z_co=-1, nu_counter=1, nu_co=2)
print("{:.3f} mol/L".format(Cco))

0.473 mol/L


Note that if we pass an invalid set of stoichiometric parameters, we'll get an `AssertionError`

In [5]:
from membrane_toolkit.core import donnan_equilibrium
# for MgCl2, we need to supply additional arguments z_counter = +2, nu_co = 2, z_co = -1, and nu_co = 2
Cs = 0.5
Cfix = 4.0

Cco = donnan_equilibrium(Cs, Cfix, z_counter=2, z_co=-2, nu_counter=1, nu_co=2)
print("{:.3f} mol/L".format(Cco))

AssertionError: 

### Units-aware computation

membrane-toolkit can automatically recognize and convert different units, by utilizing the [pint](pint.readthedocs.io/) module. **Every core function in membrane-toolkit is available with support for units-aware computation**. To use the "unitized" version of any core function, simply import it from `membrane_toolkit.core.unitized` instead of `membrane_toolkit.core`

In [6]:
from membrane_toolkit.core.unitized import donnan_equilibrium, ureg

Instead of `floats`, units-aware functions accept pint `Quantity` objects as inputs. `Quantity` objects are defined with a simple strings representing units, like 'mol/L'

In [7]:
Cs = ureg.Quantity('500 mmol/L')
Cfix = ureg.Quantity('4 mol/L')
print("Cs = {}".format(Cs))
print("Cfix = {}".format(Cfix))

Cs = 500.0 millimole / liter
Cfix = 4.0 mole / liter


Let's repeat the calculation using the unitized version of `donnan_equilibrium`. Notice that `Cs` has units of mmol/L while `Cfix` has units of mol/L. Normally we would need to manually convert these into consistent units, but the unitized version of `donnan_equilibrium` does that for us:

In [8]:
from membrane_toolkit.core.unitized import donnan_equilibrium
Cco = donnan_equilibrium(Cs, Cfix)
print("{:.3f}".format(Cco))

0.062 mole / liter


### Uncertainty propagation

Uncertainty propagation can be handled automatically using the [uncertainties](https://pythonhosted.org/uncertainties/) package. Simply create a `ufloat` instead of a regular float, and membrane-toolkit will accurately propagate the uncertainty through most calculations. In this example, we'll calculate the apparent permselectivity of a membrane, based on a measured membrane potential of 35 +/- 0.2 mV.

In [9]:
from uncertainties import ufloat

Emem = ufloat(35, 0.2)
print("Emem = {:.1f}".format(Emem))

Emem = 35.0+/-0.2


We also need avalue of the ideal membrane potential, which we can calculate with the `nernst_potential` method. In this example, we'll assume that the membrane was measured in between 0.5 M and 0.1 M NaCl solutions.

In [10]:
from membrane_toolkit.core import apparent_permselectivity, nernst_potential

In [11]:
C_high = 0.5
C_low = 0.1
E_ideal = nernst_potential(C_high, C_low) * 1000 # convert V to mV
print("E_ideal = {:.4f}".format(E_ideal))

E_ideal = 41.3485


In [12]:
from uncertainties import ufloat

Emem = ufloat(35, 0.2)
print("Emem = {:.1f}".format(Emem))

Emem = 35.0+/-0.2


In [13]:
E_ideal = 41.3485
permselectivity = apparent_permselectivity(Emem, E_ideal)
print("alpha = {}".format(permselectivity))

alpha = 0.846+/-0.005
